# Documentation for general noise maps creation

**Disclaimer:** This is not working yet.

To create our polytope we can use what we are already using. The insphere that is going to change. 

### Importing the necessary libraries

In [62]:
#Construction:
import picos as pic
import numpy as np
import math
from scipy.spatial import ConvexHull

### Creating the noiseless polytope

Here we will give the vertices of our polyhedron and ask for the ConvexHull function to find the COnvex Hull associated to this.

In [78]:
# Creating the smallest test polyhedron: Octahedron
vert = np.array([[1,1,0,0],[1,-1,0,0],[1,0,1,0],[1,0,-1,0],[1,0,0,1],[1,0,0,-1]])

#print(vert)
# Construct the polyhedron
hull = ConvexHull(vert, qhull_options = "QJ Pp")
facets = hull.equations

print(facets)

# Without forth dimension

vert_3D = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]])

# Construct the polyhedron
hull_3D = ConvexHull(vert_3D)
facets_3D = hull_3D.equations

print(facets_3D)

[[ 1.00000000e+00  4.27138325e-11 -5.44437828e-11  5.76947379e-11
  -5.00000000e-01]
 [-1.00000000e+00 -5.28765920e-12  9.18699561e-11 -2.02685646e-11
   5.00000000e-01]
 [ 1.00000000e+00 -3.21385141e-11 -5.44437828e-11  5.76947379e-11
  -5.00000000e-01]
 [-1.00000000e+00 -5.28765920e-12  1.70176095e-11 -2.02685646e-11
   5.00000000e-01]
 [ 1.00000000e+00  4.27138325e-11 -5.44437828e-11  2.44528842e-11
  -5.00000000e-01]
 [-1.00000000e+00 -5.28765920e-12  9.18699561e-11 -6.18790574e-11
   5.00000000e-01]
 [ 1.00000000e+00 -3.21385141e-11 -5.44437828e-11  2.44528842e-11
  -5.00000000e-01]
 [-1.00000000e+00 -5.28765920e-12  1.70176095e-11 -6.18790574e-11
   5.00000000e-01]]
[[-0.57735027 -0.57735027 -0.57735027 -0.28867513]
 [ 0.57735027 -0.57735027 -0.57735027 -0.28867513]
 [-0.57735027 -0.57735027  0.57735027 -0.28867513]
 [ 0.57735027 -0.57735027  0.57735027 -0.28867513]
 [ 0.57735027  0.57735027 -0.57735027 -0.28867513]
 [-0.57735027  0.57735027 -0.57735027 -0.28867513]
 [ 0.57735027

### Creating the noise map
Given the noise map
$$\Phi^\eta (A_a) = \eta A_a+(1-\eta)\text{Tr}(\xi A_a)\mathbb{1}\equiv A_a^\eta$$
it is necessary to choose the density matrix $\xi$, which defines the map. In [Hirsch2016](https://arxiv.org/abs/1512.00262) they have observed that the best choice of $\xi$ depends on the input state and conjecture that the optimal choice of $\xi$ is setting $\xi = \text{tr}_B(\rho_{\text{sep}})$. When considering a general noise map $\Phi^\eta$, $\xi$ will be an arbitrary qubit state $\xi = (\mathbb{1}+\vec{u}\cdot\vec{\sigma})/2$ .

In [64]:
u = np.array([0,0,0])

# Horodecki = (0,0,1)

xi = np.array([[1+u[2],u[0]-u[1]],[u[0]+u[1],1-u[2]]])/2

print(xi)

[[0.5 0. ]
 [0.  0.5]]


### Finding the shrinking factor

We want 
$$\eta^{*}\left(\left\lbrace A_{a|x}\right\rbrace,\xi_{A}\right) = \underset{j}{\text{min}} ~\eta^{*}_{j}\left(\vec{F}_{j},b_{j},\xi_{A}\right), \qquad j = 1,\ldots,N_{F}.$$

We will obtain a quadratic expression for $\eta(\vec{F},b,\xi_{A})$, namely,
$$A\eta^{2} + B\eta + C = 0,$$
with
$$A = \sum_{k=1}^{3}\left(F_{k} {-} F_{0} u_{k}\right)^{2}$$
$$B = 4F_{0}\sum_{k=1}^{3}u_{k}\left(F_{k} {-} F_{0}u_{k}\right)$$
$$C = 4b\left(F_{0} {-} b\right),$$
$\eta^{*}$ being the largest of the two solutions to this equation and $N_F$ being the number of facets.

In [75]:
# Here we will calculate A, B and C

# We hope to find this radius for de despolarizing map
r = np.min(np.abs(facets_3D[:, -1]))
print(r)

eta = np.zeros(facets.shape[0])

#print(facets)

for j in range(facets.shape[0]):

    F_0 = facets[j,0]
    F_1 = facets[j,1]
    F_2 = facets[j,2]
    F_3 = facets[j,3]
    b = facets[j,4]

    A = (F_1-F_0*u[0])**2+(F_2-F_0*u[1])**2+(F_3-F_0*u[2])**2
    B = 4*F_0*(u[0]*(F_1-F_0*u[0])+u[1]*(F_2-F_0*u[1])+u[2]*(F_3-F_0*u[2]))
    C = 4*b*(F_0-b)

    etas = np.array([(-B+np.sqrt(B**2-4*A*C))/(2*A),(-B-np.sqrt(B**2-4*A*C))/(2*A)])
    eta[j] = max(etas)

print(min(eta))

0.5773502691896258
25506110856.210247
